In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
import time
import os
import re

# wd = webdriver.Chrome(executable_path=r"C:\Users\Jedrek\Documents\wonga_interest_monitoring\chromedriver.exe")
EXEC_PATH = r"C:\Users\Jedrek\Documents\wonga_interest_monitoring\chromedriver.exe"
DOWNLOAD_PATH = "D:\\dokumenty\\KLAN\\"

In [7]:
def set_up_webdriver(exec_path, download_path):
    path = os.path.abspath(download_path)
    prefs = {"download.default_directory":path}
    options = Options()
    options.add_experimental_option("prefs", prefs)
    wd = webdriver.Chrome(executable_path=exec_path, options = options)
    return wd


In [8]:
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [9]:
def download_wait(directory, timeout, expected):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        
        ep_number_found = []
        for fname in files:
            for number in expected:
                if number in fname:
                    ep_number_found.append(number)
        dl_wait = not checkEqual(expected, ep_number_found)
        seconds += 1
        set_E = set(expected)
        set_F = set(ep_number_found)
        if seconds % 15 == 0:
            print("Expected, but not found: ", set_E.difference(set_F))
            print("Found, but unexpected: ", set_F.difference(set_E))
    return seconds

In [10]:
def download_season(Season_number):
    start = time.time()
    ## THE PATH TO DOWNLOAD INTO
    download_path_s = DOWNLOAD_PATH + str(Season_number)
    
    ## LOOK INTO THE DOWNLOAD PATH, MAYBE SOME EPISODES ARE ALREADY THERE
    files = os.listdir(download_path_s)
    downloaded_episodes = [str(int(re.findall(r'E(\d{4}|\d{3})', i)[0])) for i in files]
    
    ## WEBDRIVER SETUP
    wd = set_up_webdriver(EXEC_PATH, download_path_s)
    
    ## request and transform the list of downloads
    page = f'http://staryklan.cba.pl/sezon{Season_number}.html'
    r = requests.get(page)
    txt = r.text
    soup = BeautifulSoup(txt, 'html.parser')
    li = soup.get_text().split('Odcinek')
    li2 = [i.strip().replace('\n', '') for i in li][1:]
    di = dict([i.split(":",1) for i in li2])
    
    ## initialize loop
    counter = 0
    logger = dict.fromkeys(di.keys())
    for key in di:
            if key[:3] not in downloaded_episodes:
                wd.switch_to.window(wd.window_handles[counter])
                url = di[key][:73] # hack, this is the proper length of download link
                try:
                    wd.get(url)
                    btn = ui.WebDriverWait(wd, 30).until(lambda wd: wd.find_element_by_xpath('//*[@id="startholder"]/div[2]/div[2]/section/div[1]/div/div[4]/div[3]/div[2]/div[1]/button'))
                    btn.click()
                    logger[key] = 'CLICKED'
                except BaseException as e:
                    logger[key] = e

                wd.execute_script("window.open(''),'_blannk'")
                counter += 1
                print(f'Odcinek {key}')
            else:
                print(f"Odcinek {key} has already been downloaded!")
            
            if counter % 3 == 0 and counter != 0:               
                
                nap = 200
                
                print(f'For a maximum of {nap} seconds I will be checking how the download is doing...')

                
                clicked_episodes = [k[:3] for k,v in logger.items() if v == 'CLICKED']
                clicked_or_downloaded = clicked_episodes + downloaded_episodes 
                
                elapsed = download_wait(download_path_s, nap, clicked_or_downloaded)
                
                if elapsed == nap:
                    print(f'The download is taking too long, something is wrong here...')
                else:
                    print(f'Slept for {elapsed}, back to work')
                    
                time.sleep(5)
                wd.quit()
                wd = set_up_webdriver(EXEC_PATH, download_path_s)
                counter = 0
                       
    end = time.time()
    print(f'Phew! That was a lot of work, it took {end - start}')
    
    return di, logger


In [11]:
season_dict_4, logger_4 = download_season(4)

Odcinek 364 has already been downloaded!
Odcinek 365 has already been downloaded!
Odcinek 366 has already been downloaded!
Odcinek 367 has already been downloaded!
Odcinek 368-369 has already been downloaded!
Odcinek 370 has already been downloaded!
Odcinek 371 has already been downloaded!
Odcinek 372 has already been downloaded!
Odcinek 373 has already been downloaded!
Odcinek 374 has already been downloaded!
Odcinek 375 has already been downloaded!
Odcinek 376-377 has already been downloaded!
Odcinek 378 has already been downloaded!
Odcinek 379 has already been downloaded!
Odcinek 380 has already been downloaded!
Odcinek 381 has already been downloaded!
Odcinek 382 has already been downloaded!
Odcinek 383 has already been downloaded!
Odcinek 384 has already been downloaded!
Odcinek 385 has already been downloaded!
Odcinek 386 has already been downloaded!
Odcinek 387 has already been downloaded!
Odcinek 388 has already been downloaded!
Odcinek 389 has already been downloaded!
Odcinek 